In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tqdm import tqdm

In [49]:
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
from IPython.display import Image, display
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split

In [50]:
image_dir = Path('../input/vcor-vehicle-color-recognition-dataset/train')

# Get filepaths and labels
filepaths = list(image_dir.glob(r'*/*'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
train_df = pd.concat([filepaths, labels], axis=1)

In [51]:
categories = list(train_df['Label'].unique())
traindir = '../input/vcor-vehicle-color-recognition-dataset/train'
testdir  = '../input/vcor-vehicle-color-recognition-dataset/test'

In [52]:
print(categories, len(categories))

In [53]:
for category in categories:
    path=os.path.join(traindir, category)
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img))
        plt.imshow(img_array)
        plt.show()
        break
    break

In [54]:
def create_data(data_list,datadir):
    for category in categories:
        path=os.path.join(datadir, category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(100,100))
                data_list.append([new_array,class_num])
            except Exception as e:
                pass            

In [55]:
train_data = []
test_data = []
create_data(train_data, traindir)
create_data(test_data, testdir)

In [56]:
train_data[0]

In [57]:
print(len(train_data),len(test_data))

In [58]:
def create_splits(data, length):
    X = []
    y = []
    for categories, label in data:
        X.append(categories)
        y.append(label)
    X = np.array(X).reshape(length,-1)
    y = np.array(y)
    return X,y


In [59]:
x_train, y_train = create_splits(train_data, len(train_data))
x_test, y_test = create_splits(test_data, len(test_data))

In [60]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [61]:
x_train = x_train/255
x_test = x_test/255

In [62]:
x_train[1]

In [63]:
from sklearn.svm import SVC
svc = SVC(kernel='linear',gamma='auto')
svc.fit(x_train, y_train)

In [64]:
y2 = svc.predict(x_test)

In [65]:
from sklearn.metrics import accuracy_score
print("Accuracy on unknown data is",accuracy_score(y_test,y2))

In [66]:
from sklearn.metrics import classification_report
print("Accuracy on unknown data is",classification_report(y_test,y2))

In [67]:
result = pd.DataFrame({'original' : y_test,'predicted' : y2})

In [68]:
result
